In [1]:
#Necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
#Filter warnings
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
#Read data and change categoric values
df = pd.read_csv("Hitters.csv")

df = df.dropna()

dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

y = df["Salary"]

X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype("float64")

X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

Model and Prediction

In [4]:
#Scaling the train data because ANN works better with homogeneous datasets
scaler = StandardScaler()

In [6]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [7]:
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [8]:
mlp_model = MLPRegressor().fit(X_train_scaled, y_train)

In [11]:
#Predict the test data
y_pred = mlp_model.predict(X_test_scaled)

In [12]:
#RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

660.4093656232483

Model Tuning

In [13]:
mlp_params = {"alpha" : [0.1, 0.01, 0.02, 0.001, 0.0001], 
              "hidden_layer_sizes": [(10,20), (5,5), (100,100)]}

In [15]:
mlp_cv = GridSearchCV(mlp_model, mlp_params, cv = 10, n_jobs=-1, verbose=2).fit(X_train_scaled,y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


In [16]:
mlp_cv.best_params_

{'alpha': 0.02, 'hidden_layer_sizes': (100, 100)}

In [17]:
#Prediction with tuned model
mlp_tuned = MLPRegressor(alpha=0.02, hidden_layer_sizes=(100,100)).fit(X_train_scaled,y_train)

In [18]:
y_pred_tuned = mlp_tuned.predict(X_test_scaled)

In [19]:
#RMSE

rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

rmse_tuned

353.29810526583753